In [1]:
fpath = '/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/output/T5_base_prefix_text2cypher_with_cell_value/predictions_eval_72.72544080604534.json'

In [37]:
schema_fpath = '/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/data/text2cypher/cased/schema.json'
import json
with open(schema_fpath, 'r', encoding="utf-8") as f:
    schema = json.load(f)
    print(len(schema))


155


In [38]:
def get_uncased_schema(schema):
    uncased_schema = {}
    for db, cont in schema.items():
        uncased_schema[db.lower()]={}
        for k, v in cont.items():
            if isinstance(v, str):
                uncased_schema[db.lower()][k]=v.lower()
            if isinstance(v, list):
                uncased_schema[db.lower()][k]=[item.lower() if isinstance(item, str) else item for item in v ]
            if isinstance(v, dict):
                uncased_schema[db.lower()][k.lower()] = {}
                for kk, vv in v.items():
                    uncased_schema[db.lower()][k.lower()][kk.lower()] = vv
    for db, cont in uncased_schema.items():
        for k, v in cont.items():
            if k=='property_names':
                cased_property_names = v
                for i, each in enumerate(cased_property_names):
                    if isinstance(each, list) and i !=0:
                        for j, element in enumerate(each):
                            if isinstance(element, str):
                                uncased_schema[db][k][i][j]= element.lower()  
    return uncased_schema


In [39]:
uncased_schema = get_uncased_schema(schema)

In [35]:

from py2neo import Graph
import configparser
config = configparser.ConfigParser()
config.read('/home/22842219/Desktop/phd/SemanticParser4Graph/application/config.ini')
filenames = config["FILENAMES"]

neo4j_uri = filenames['neo4j_uri']
neo4j_user = filenames['neo4j_user']
neo4j_password = filenames['neo4j_password']
graph = Graph(neo4j_uri, auth = (neo4j_user, neo4j_password))

In [41]:
from evaluator import Evaluator, print_scores
from py2neo.data import Node, Relationship
test = 'match (singer:`concert_singer.singer`) return count(*)'
prediction_res = graph.run(test).data()
# print(prediction_res)
p_res=[]
for dict_p in prediction_res:
    for k, v in dict_p.items():
        if isinstance(v, Node) or isinstance(v, Relationship):
            continue
        else:
            p_res.append(tuple(dict_p.values()))
            print(p_res)

[(6,)]


In [42]:
#text2cypher
code_t5_finetune_post_all = '/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/output/CodeT5_base_finetune_text2cypher-predicted_postprocessing_removing_space-normalization/predictions_predict.json'
code_t5_prefix_post_all = '/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/output/CodeT5_base_prefix_text2cypher--predicted_post_all/predictions_predict.json'
t5_prefix_post_all= '/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/output/T5_base_prefix_text2cypher_with_cell_value-predicted_postprocessing_removing_space-normalization/predictions_predict.json'
t5_finetune_post_all  ='/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/output/T5_base_finetune_text2cypher_with_cell_value-predicted_postprocessing_removing_space-normalization/predictions_predict.json'

# [code_t5_finetune_post_all, code_t5_prefix_post_all,  t5_prefix_post_all,  t5_finetune_post_all]
import json
from evaluator import Evaluator, print_scores

for f in [code_t5_finetune_post_all, code_t5_prefix_post_all,  t5_prefix_post_all,  t5_finetune_post_all]:
    preds = []
    golds = []
    with open(f, 'r', encoding="utf-8") as f:
        data = json.load(f)
        print(len(data))
        for each in data:
            # print(each)
            pred = each['prediction']
            gold = each['query']
            preds.append(pred)
            golds.append(gold)
    evaluator = Evaluator(graph, 'all')
    results = []
    for predicted, gold in zip(preds, golds):
        results.append(evaluator.evaluate_one(gold, predicted))
    evaluator.finalize()

    print_scores(evaluator.scores, 'all')
    rest =  {
        "per_item": results,
        "total_scores": evaluator.scores,
    }

491
preds: match (singer:`concert_singer.singer`) return count(*)
q_res: [(6,)]
p_res: [(6,)]
sorted results. p_sorted: [(6,)], q_sorted: [(6,)]
preds: match (singer:`concert_singer.singer`) return count(*)
q_res: [(6,)]
p_res: [(6,)]
sorted results. p_sorted: [(6,)], q_sorted: [(6,)]
preds: match (singer:`concert_singer.singer`) return singer.name,singer.country,singer.age order by singer.age desc
q_res: [('Joe Sharp', 'Netherlands', 52), ('John Nizinik', 'France', 43), ('Rose White', 'France', 41), ('Timbaland', 'United States', 32), ('Justin Brown', 'France', 29), ('Tribal King', 'France', 25)]
p_res: [('Joe Sharp', 'Netherlands', 52), ('Joe Sharp', 'Netherlands', 52), ('Joe Sharp', 'Netherlands', 52), ('John Nizinik', 'France', 43), ('John Nizinik', 'France', 43), ('John Nizinik', 'France', 43), ('Rose White', 'France', 41), ('Rose White', 'France', 41), ('Rose White', 'France', 41), ('Timbaland', 'United States', 32), ('Timbaland', 'United States', 32), ('Timbaland', 'United State

In [ ]:
pipeline2_rest='/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/metrics/text2cypher/two_steps_finetune.json'
kaya_t5_prefix_post_all = '/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/kaya/output/T5_base_prefix_text2cypher_with_cell_value-postprocessing_all/predictions_predict.json'
kaya_t5_finetune_post_all = '/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/kaya/output/T5_base_finetune_text2cypher_with_cell_value/predictions_predict.json'
